## Installing and loading the libraries

In [1]:
!pip install git+https://github.com/huggingface/transformers #use this only for codellama

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-azu0j42t
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-azu0j42t
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - \ done
    Preparing wheel metadata ... - \ done
     |████████████████████████████████| 3.8 MB 4.4 MB/s eta 0:00:01
  Using cached huggingface_hub-0.18.0-py3-none-any.whl (301 kB)
  Created wheel for transformers: filename=transformers-4.35.0.dev0-py3-none-any.whl size=7884158 sha256=ca500c64fe6e3bbe9e46bd4516dd2989d51bf6b89eec17a4629ad4c884aa4048
  Stored in directory: /tmp/pip-ephem-wheel-cache-hgzekkib/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers
ERROR: tokenizers 0.14.1 has requirement huggingface_hub<0.18,>=0.16.4, but you'll have huggingface-hub 0.18.0 which is incompatible.
ERROR: azureml-automl-dnn-nlp 1.51.0 has requirement dataset

In [2]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
Requirement already up-to-date: datasets==2.13.0 in /anaconda/envs/azureml_py38/lib/python3.8/site-packages (2.13.0)
Requirement already up-to-date: peft==0.4.0 in /anaconda/envs/azureml_py38/lib/python3.8/site-packages (0.4.0)
Requirement already up-to-date: accelerate==0.21.0 in /anaconda/envs/azureml_py38/lib/python3.8/site-packages (0.21.0)
Requirement already up-to-date: bitsandbytes==0.40.2 in /anaconda/envs/azureml_py38/lib/python3.8/site-packages (0.40.2)
Requirement already up-to-date: trl==0.4.7 in /anaconda/envs/azureml_py38/lib/python3.8/site-packages (0.4.7)
Requirement already up-to-date: safetensors>=0.3.1 in /anaconda/envs/azureml_py38/lib/python3.8/site-packages (0.4.0)
     |████████████████████████████████| 7.8 MB 5.6 MB/s eta 0:00:01
ERROR: azureml-automl-dnn-nlp 1.51.0 has requirement datasets<=2.3.2,>=1.7.0, but you'll have datasets 2.13.0 which is incompatible.
ERROR: azureml-automl-dnn-nlp 1.51.0 has r

In [3]:
!pip install ipywidgets==7.7.1
!pip install huggingface_hub
!pip install python-dotenv
!pip install tiktoken

     |████████████████████████████████| 123 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 58.1 MB/s eta 0:00:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 4.0.9
    Uninstalling widgetsnbextension-4.0.9:
      Successfully uninstalled widgetsnbextension-4.0.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 8.1.1
    Uninstalling ipywidgets-8.1.1:
      Successfully uninstalled ipywidgets-8.1.1


In [4]:
from datasets import load_dataset
from random import randrange

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM

from trl import SFTTrainer

# Data Preparation

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("amk_qa_code_explanations.csv")
df.head()

FileNotFoundError: ignored

In [ ]:
df.shape

In [ ]:
df.drop_duplicates(['scenario','step_function','page_function'], inplace=True)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df[df['page_function']=="no page definition for the given scenario"].shape

In [ ]:
#clean the scenario from spl characters
import re
df['scenario'] = df['scenario'].apply(lambda x: re.sub(r'(@When|@Then|@And|@Given|\"|\^|\$)', '', x))
df['scenario'] = df['scenario'].apply(lambda x: x.strip('()'))

In [ ]:
df['scenario'].loc[2]

In [ ]:
step_instruction = "### Instruction: Use the Task below and the Input given to write the Response, which is a Java programming code that can solve the following Task:\n\n### Task: Write automation script for UI Testing that contains only a step definition function, written using Cucumber and Selenium frameworks, for the following input:"
page_instruction = "### Instruction: Use the Task below and the Input given to write the Response, which is a Java programming code that can solve the following Task: \n\n### Task: Write automation script for UI Testing that contains only a page object definition function, written using Cucumber and Selenium frameworks, for the following input:"

In [ ]:
#prepare a prompt
df['step_inst'] = step_instruction + "\n\n### Input:\n" + df['scenario'] + "\n\n### Output:\n" + df['step_function']
df['page_inst'] = page_instruction + "\n\n### Input:\n" + df['scenario'] + "\n\n### Output:\n" + df['page_function']

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
print(df['step_inst'].loc[0])

In [ ]:
print(df['page_inst'].loc[0])

In [ ]:
df_new = pd.DataFrame(pd.concat([df['step_inst'],df['page_inst']]), columns=['text'])
df_new.head()

In [ ]:
df_new.shape

In [ ]:
df_new.reset_index(drop=True, inplace=True)

In [ ]:
print(df_new.loc[0, 'text'])

In [ ]:
# Shuffle the rows
shuffled_df = df_new.sample(frac=1, random_state=42)

In [ ]:
# Split the DataFrame into train and test sets
from sklearn.model_selection import train_test_split
df_train, df_valid = train_test_split(shuffled_df, test_size=74, random_state=42)
print(df_train.shape, df_valid.shape)

In [ ]:
df_train.reset_index(drop=True, inplace=True)
df_valid.reset_index(drop=True, inplace=True)

In [ ]:
df_train.to_csv("train.csv", index=False)
df_valid.to_csv("valid.csv", index=False)

## Connect to Huggingface Hub

You can log in to Hugging Face Hub interactively

In [8]:
from huggingface_hub import notebook_login
# Log in to HF Hub
notebook_login()

Or you can provide .env file containing the Hugging Face token

In [ ]:
# from huggingface_hub import login
# from dotenv import load_dotenv
# import os

# # Load the enviroment variables
# load_dotenv()
# # Login to the Hugging Face Hub
# login(token=os.getenv("HF_HUB_TOKEN"))

# Load the dataset

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [5]:
# Load dataset from the hub
hf_data_repo = "shyam-incedoinc/qa-finetune-dataset"
train_dataset = load_dataset(hf_data_repo, split="train")
valid_dataset = load_dataset(hf_data_repo, split="validation")

# Show dataset size
print(f"dataset size: {len(train_dataset), len(valid_dataset)}")

# Show an example
print(train_dataset[randrange(len(train_dataset))])


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/shyam-incedoinc___csv/shyam-incedoinc--qa-finetune-dataset-14ea1221b860178c/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


dataset size: (400, 74)
{'text': '### Instruction: Use the Task below and the Input given to write the Response, which is a Java programming code that can solve the following Task:\n\n### Task: Write automation script for UI Testing that contains only a step definition function, written using Cucumber and Selenium frameworks, for the following input:\n\n### Input:\nI (dont )?see default text (.*) in email or mobile textfield\n\n### Output:\npublic void validateDefaultTextEmailOrMobileTextfield(String dontFlag, String value) {\n\t\tloginFirstScreenPage.validateDefaultTextEmailOrMobileTextfield(dontFlag,value);\n\t}'}


# Fine-Tuning

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
# Get the type
compute_dtype = getattr(torch, "float16")

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)

# device_map = {"": 0}

In [7]:
%%time
model_id = "szlay/seleniumai-llama2-7b-v1.1"

# Load the pretrained model
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config,
                                             use_cache = False, device_map="auto")
model.config.pretraining_tp = 1

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

CPU times: user 24.4 s, sys: 30.4 s, total: 54.8 s
Wall time: 2min 50s


In [8]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)
# Not necessary when using SFTTrainer
# prepare model for training
# model = prepare_model_for_kbit_training(model)
# model = get_peft_model(model, peft_config)

In [9]:
# Define the training arguments. For full list of arguments, check
#https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments
args = TrainingArguments(
    output_dir='llama2-7b-selenium-peft-qlora-finetuned-qa',
    num_train_epochs=3,
    per_device_train_batch_size=2, # 6 if use_flash_attention else 4,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    #save_steps=save_steps,
    logging_steps=25,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    load_best_model_at_end=True,
    greater_is_better=False,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    #max_steps=max_steps,
    group_by_length=False,
    lr_scheduler_type="cosine",
    disable_tqdm=False,
    report_to="tensorboard",
    seed=42
)

In [10]:
# Create the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    dataset_text_field='text',
    peft_config=peft_config,
    max_seq_length=2048,
    tokenizer=tokenizer,
    packing=True,
    args=args,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [11]:
# # train
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


OutOfMemoryError: ignored

In [ ]:
# save model in local
trainer.save_model()

# Merge the model and the adapters and save it

When running in a T4 instance we have to clean the memory

In [ ]:
# Empty VRAM
del model
del trainer
import gc
gc.collect()
gc.collect()

20013

In [ ]:
torch.cuda.empty_cache()

In [ ]:
gc.collect()

0

Reload the trained and saved model and merge it then we can save the whole model

In [ ]:
from peft import AutoPeftModelForCausalLM

new_model = AutoPeftModelForCausalLM.from_pretrained(
    'codegen25-7b-multi-peft-qlora-finetuned-qa',
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="balanced",
)

# Merge LoRA and base model
merged_model = new_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("codegen25-7b-multi-peft-qlora-finetuned-qa_merged",safe_serialization=True)
tokenizer.save_pretrained("codegen25-7b-multi-peft-qlora-finetuned-qa_merged")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

('codegen25-7b-multi-peft-qlora-finetuned-qa_merged/tokenizer_config.json',
 'codegen25-7b-multi-peft-qlora-finetuned-qa_merged/special_tokens_map.json',
 'codegen25-7b-multi-peft-qlora-finetuned-qa_merged/added_tokens.json')

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
#use the write token
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# push merged model to the hub
hf_model_repo = "shyam-incedoinc/codegen25-7b-multi-peft-qlora-finetuned-qa"
merged_model.push_to_hub(hf_model_repo)
tokenizer.push_to_hub(hf_model_repo)

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.84G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/shyam-incedoinc/codegen25-7b-multi-peft-qlora-finetuned-qa/commit/65753edff280b723002097de3d320cee71a4f695', commit_message='Upload tokenizer', commit_description='', oid='65753edff280b723002097de3d320cee71a4f695', pr_url=None, pr_revision=None, pr_num=None)

# Test the merged model

In [ ]:
sample = valid_dataset[randrange(len(valid_dataset))]['text']
groundtruth = sample.split("### Output:\n")[1]
print(groundtruth)
prompt = sample.split("### Output:\n")[0]+"### Output:\n"
print(prompt)

public void Project(String projectName)
    {
        adminLoginPage.assignProject(projectName);
    }
### Instruction: Use the Task below and the Input given to write the Response, which is a Java programming code that can solve the following Task:

### Task: Write automation script for UI Testing that contains only a step definition function, written using Cucumber and Selenium frameworks, for the following input:

### Input:
I assign (.*) Project

### Output:



In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=1024,
                                do_sample=True, top_p=0.9, temperature=0.6)

# print(f"Prompt:\n{prompt}\n")
print(f"Generated response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")#[0][len(prompt):]}")
print(f"Ground Truth:\n{groundtruth}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Generated response:
### Instruction: Use the Task below and the Input given to write the Response, which is a Java programming code that can solve the following Task:

### Task: Write automation script for UI Testing that contains only a step definition function, written using Cucumber and Selenium frameworks, for the following input:

### Input:
I assign (.*) Project

### Output:
public void AssignProjectToCustomer(String projectName) throws Exception
    {
    	userManagementPage.AssignProjectToCustomer(projectName);
    }
Ground Truth:
public void Project(String projectName)
    {
        adminLoginPage.assignProject(projectName);
    }


In [ ]:
sample = valid_dataset[randrange(len(valid_dataset))]['text']
groundtruth = sample.split("### Output:\n")[1]
print(groundtruth)
prompt = sample.split("### Output:\n")[0]+"### Output:\n"
print(prompt)

public void uploadLmiDocument(String file, String value) throws AWTException, IOException {
		if((file!=null)&&(file.contains("data file")))
			value=commonPage.fetchDataFromDataProperties(value);
			dataHealthPagePage.uploadDocument(value);
	}
### Instruction: Use the Task below and the Input given to write the Response, which is a Java programming code that can solve the following Task:

### Task: Write automation script for UI Testing that contains only a step definition function, written using Cucumber and Selenium frameworks, for the following input:

### Input:
After fetching data from data file )?I upload (.*) Utility bill document

### Output:



In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=1024,
                                do_sample=True, top_p=0.9, temperature=0.6)

# print(f"Prompt:\n{prompt}\n")
print(f"Generated response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")#[0][len(prompt):]}")
print(f"Ground Truth:\n{groundtruth}")

Generated response:
### Instruction: Use the Task below and the Input given to write the Response, which is a Java programming code that can solve the following Task:

### Task: Write automation script for UI Testing that contains only a step definition function, written using Cucumber and Selenium frameworks, for the following input:

### Input:
After fetching data from data file )?I upload (.*) Utility bill document

### Output:
public void uploadUtilityBillDocument(String file, String value) throws AWTException, IOException {
		if((file!=null)&&(file.contains("data file")))
			value=commonPage.fetchDataFromDataProperties(value);
		utilityBillPage.uploadUtilityBillDocument(value);
	}
Ground Truth:
public void uploadLmiDocument(String file, String value) throws AWTException, IOException {
		if((file!=null)&&(file.contains("data file")))
			value=commonPage.fetchDataFromDataProperties(value);
			dataHealthPagePage.uploadDocument(value);
	}


In [ ]:
sample = valid_dataset[randrange(len(valid_dataset))]['text']
groundtruth = sample.split("### Output:\n")[1]
print(groundtruth)
prompt = sample.split("### Output:\n")[0]+"### Output:\n"
print(prompt)

public void selectRole(int num) throws InterruptedException {
		String role=String.format(parameterizedRole, num);
		WebElement elem=getDriver().findElement(By.xpath(role));
		WebDriverWait wait=new WebDriverWait(getDriver(),15);
		wait.until(ExpectedConditions.elementToBeClickable(elem));
		Thread.sleep(2000);
		clickOn(elem);
	}
### Instruction: Use the Task below and the Input given to write the Response, which is a Java programming code that can solve the following Task: 

### Task: Write automation script for UI Testing that contains only a page object definition function, written using Cucumber and Selenium frameworks, for the following input:

### Input:
I select the role user

### Output:



In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=1024,
                                do_sample=True, top_p=0.9, temperature=0.6)

# print(f"Prompt:\n{prompt}\n")
print(f"Generated response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")#[0][len(prompt):]}")
print(f"Ground Truth:\n{groundtruth}")

Generated response:
### Instruction: Use the Task below and the Input given to write the Response, which is a Java programming code that can solve the following Task: 

### Task: Write automation script for UI Testing that contains only a page object definition function, written using Cucumber and Selenium frameworks, for the following input:

### Input:
I select the role user

### Output:
public void clickOnLogin() {
		loginButton.click();
	}
Ground Truth:
public void selectRole(int num) throws InterruptedException {
		String role=String.format(parameterizedRole, num);
		WebElement elem=getDriver().findElement(By.xpath(role));
		WebDriverWait wait=new WebDriverWait(getDriver(),15);
		wait.until(ExpectedConditions.elementToBeClickable(elem));
		Thread.sleep(2000);
		clickOn(elem);
	}


# Load the model from the HF Hub and test it

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

hf_model_repo = "shyam-incedoinc/codellama-7b-hf-peft-qlora-finetuned-qa" #"shyam-incedoinc/codegen25-7b-multi-peft-qlora-finetuned-qa"

# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(hf_model_repo, trust_remote_code=True)

# Load the model
model = AutoModelForCausalLM.from_pretrained(hf_model_repo,
                                             load_in_4bit=True,
                                             torch_dtype=torch.float16,
                                             device_map="balanced")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Load the sample
sample = valid_dataset[randrange(len(valid_dataset))]['text']
groundtruth = sample.split("### Output:\n")[1]
prompt = sample.split("### Output:\n")[0]+"### Output:\n"
# print(prompt)

In [ ]:
# Generate response
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=2048,
                         do_sample=True, top_p=0.9, temperature=0.6)

# Print the result
print(f"Generated response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")#[0][len(prompt):]}")
print(f"Ground Truth:\n{groundtruth}")

Generated response:
### Instruction: Use the Task below and the Input given to write the Response, which is a Java programming code that can solve the following Task:

### Task: Write automation script for UI Testing that contains only a step definition function, written using Cucumber and Selenium frameworks, for the following input:

### Input:
After fetching data from data file )?I see prepopulated value (.*) in email or mobile textfield in Signup

### Output:
public void validatePrepopulatedEmailOrMobileTextfield(String file, String value) throws IOException {
    	if((file!=null)&&(file.contains("data file")))
			value=commonPage.fetchDataFromDataProperties(value);
    	signUpFirstPage.validatePrepopulatedEmailOrMobileTextfield(value);
    }
Ground Truth:
public void validatePrepopulatedEmailOrMobileTextfield(String file, String value) throws FileNotFoundException, IOException {
        if((file!=null)&&(file.contains("data file")))
            value=commonPage.fetchDataFromDataPr